In [ ]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4 as nc
import geopandas as gpd
import rioxarray as rxr
from myfunc import timer
from myfunc import DirMan
from myfunc import load_and_flatten_data
import config

# configuration
# resolution = "0p1"
resolution = "500"
region = [-180,180,-60,90]
data_path = f'/tera04/zhwei/xionghui/bedrock/run/{resolution}/'
post_data_path = '/tera04/zhwei/xionghui/bedrock/'
shp_path = '/tera04/zhwei/xionghui/bedrock/Shp/'
fig_path = f'/home/xuxh22/stu01/Bedrock/fig/{resolution}/'
path = '/home/xuxh22/stu01/Bedrock/'

if resolution == "0p1":
    size = 0.1
elif resolution == "500":
    size = 0.0005

dir_man = DirMan(data_path)
dir_man.enter()

os.makedirs(f'{data_path}/csv', exist_ok=True)

# area_sum = 19.925158940748766

In [ ]:
df = pd.DataFrame()

with nc.Dataset('mask1234.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('mask1234', 'Band1'),
    ('Sbedrock', 'Sbedrock'),
    ('Area', 'area'),

    ('Dbedrock_Frequency', 'Dbedrock'),
    ('D_sum_Frequency', 'Frequency'),
    ('D_sum_duration', 'Duration'),
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)


df = df.dropna()
df = df[df['Sbedrock'] > 0]
df = df[df['mask1234'] == 1]
df['Dbedrock_Frequency'] = df['Dbedrock_Frequency'].round().astype(int)

ssf = df['D_sum_Frequency'].mean()
ssd = df['D_sum_duration'].mean()

d_f = (df.groupby('Dbedrock_Frequency')['D_sum_duration'].mean())

print(ssf)
print(ssd)
print(d_f)
print((df['Area'].sum()/1e12))

df.drop(labels='mask1234',axis=1,inplace=True)
with open('csv/Global_D_time_sum.csv','w') as f:
    df.to_csv(f)
df

In [ ]:
d_f2 = (df.groupby('Dbedrock_Frequency')['D_sum_Frequency'].mean())
print(d_f2)